# YDJ Accounting System Ver.2
##### 2021.08.22

### 1. 모듈 불러오기

In [136]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import io
import googleapiclient.http

from PIL import Image,ImageDraw,ImageFont
import os

import numpy as np
import pandas as pd
import re

import img2pdf

### 2. drive service 설정

In [138]:
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

drive_service = build('drive', 'v3', credentials=creds)

### 3. 필요한 메소드 정의

In [139]:
# 품명/수량/금액 나누는 메소드
def divide(full):
    item = full.split('\n')
    item_list = [i.split("/") for i in item]
    for i in range(len(item_list)):
        item_list[i][2] = str(commaParse(item_list[i][2]))+" 원"
    return item_list

# 사진 링크에서 id 뽑아내는 메소드
def get_id(url_list):
    url_list = [url+";" for url in url_list]
    pattern = re.compile('{}(.*?){}'.format(re.escape('id='), re.escape(';')))
    return [pattern.findall(url) for url in url_list] 

# 숫자를 000,000,000 꼴로 바꾸는 정규식
def commaParse(num):
    return re.sub('(?<=\d)(?=(\d{3})+(?!\d))',',',str(num))

### 4. 답변 CSV 전처리 for excel

In [141]:
month = int(input("월: "))
when = input('전 or 후: ')
original = pd.read_csv(f'original/use/{month}월_{when}반.csv')

df = original.copy()
df['영수증'] = get_id(df['영수증 (사진 파일로 업로드)'])
df['개수'] = [len(v) for v in df['영수증']]
df = df.drop('영수증 (사진 파일로 업로드)', axis=1)

real = sum(df['개수'][df['실물영수증 여부']=='실물영수증 O'])

df = df[['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수']]

new_df = pd.DataFrame(columns=['프로그램명', '인원', '계정사유', '적요', '결제일자', '금액', '가맹점명','개수'])

index = 0
for idx, val in df.iterrows():
    num = val['개수']
    for i in range(num):
        new_df.loc[index] = val
        index +=1
        
new_df.to_excel(f'result/use/{month}월_{when}반 예산내역서 내용.xlsx', index=False)

whole_num = sum(df['개수'])
print('영수증은 총 {}개 입니다.'.format(whole_num))
print('실물영수증은 총 {}개 입니다.'.format(real))
print('예산내역서 내용이 저장되었습니다.')

월:  8
전 or 후:  후


### 5. 답변 CSV 전처리 for 영수증 양식

In [130]:
# 불러오기
df = original.copy()

# 필요한 column만 남기기
df = df[['사용 RA','인원','결제일자','프로그램명','금액','품명/수량/금액','영수증 (사진 파일로 업로드)']]

# id만 list로 뽑아내는 과정
df['영수증'] = get_id(df['영수증 (사진 파일로 업로드)'])
df = df.drop('영수증 (사진 파일로 업로드)', axis=1)

# 원하는 꼴로 변형
df['인원'] = [str(v)+" 명" for v in df['인원']]
df['금액'] = [str(commaParse(v))+" 원" for v in df['금액']]

# 미리보기
df.head()

,사용 RA,인원,결제일자,프로그램명,금액,품명/수량/금액,영수증
0,홍나현 RA,11 명,2021-05-17,동주사진전,"12,900 원",버라이어티 초콜릿 과자 선물세트/1/12900,"[13yWWP8zPiG9cls_ChYkUhkFt1ZvNIpTy, 1c-jzznjSR..."
1,신예진 RA,14 명,2021-05-17,윤동주하우스 특별강연,"276,800 원",토끼정 카레/5/9800\n콜라/7/2000\n날치알크림파스타/3/11800\n크림...,[1sCFMJya5S9_zGoBSaHQBoZ25-8JXnJNG]
2,신예진 RA,6 명,2021-05-19,리틀포레스트: 송도편,"129,870 원",NaN,[16zZzRQEyYH4vuX3Df9e7e03TqSAzmhAW]
3,신예진 RA,6 명,2021-05-19,리틀포레스트: 송도편,"23,600 원",NaN,[1PYvuzEgVzBchgmaWkuofShXIRyWm6SzV]
4,신예진 RA,6 명,2021-05-19,리틀포레스트: 송도편,"32,200 원",NaN,[12QIk6ybAFx3JUnAAvt5jwzkYV7CqcgAp]


In [131]:
df = df.iloc[1:3]
df['업체선정사유'] = '가까워서'
df

,사용 RA,인원,결제일자,프로그램명,금액,품명/수량/금액,영수증,업체선정사유
1,신예진 RA,14 명,2021-05-17,윤동주하우스 특별강연,"276,800 원",토끼정 카레/5/9800\n콜라/7/2000\n날치알크림파스타/3/11800\n크림...,[1sCFMJya5S9_zGoBSaHQBoZ25-8JXnJNG],가까워서
2,신예진 RA,6 명,2021-05-19,리틀포레스트: 송도편,"129,870 원",NaN,[16zZzRQEyYH4vuX3Df9e7e03TqSAzmhAW],가까워서


### 6. 필요한 좌표 값 리스트

In [132]:
location_short = [(1640, 205), (2600, 205), (690,270), (1600, 270),(2555, 270)]
location_long = [(1640, 205), (2600, 205), (690,270), (1550, 270),(2555, 270)]
location_house = (690, 205)
location_receipt = (300,670)
location_reason = (550,525)
col_item = [2260, 2690, 2782]
row_item = [685, 830, 964, 1107, 1265, 1402, 1540, 1677, 1830, 1962]

### 7. Pillow 기본 설정

In [133]:
# path 설정
path = '/Users/shinyehjin/Programming/ydj-accounting-system'

# 폰트 경로와 사이즈 설정
regularFont =ImageFont.truetype(os.path.join(path,'fonts/HANBatangB.ttf'),36)
smallFont =ImageFont.truetype(os.path.join(path,'fonts/HANBatangB.ttf'),30)

### 8. 대망의 결과물 뽑아내기 과정

In [135]:
check = 0
for index in range(len(df)):
    
    target_image = Image.open(path+"/original/form.jpeg")
    row = df.iloc[index]

    draw = ImageDraw.Draw(target_image)
    draw.text(location_house, "윤동주 하우스", fill="black", font=regularFont, align='center')
    draw.text(location_reason, str(row[7]), fill="black", font=regularFont, align='center')
    if (len(row[3]) > 9):
        for i in range(5):
            draw.text(location_long[i], str(row[i]), fill="black", font=regularFont, align='center')
    else:
        for i in range(5):
            draw.text(location_short[i], str(row[i]), fill="black", font=regularFont, align='center')
    
    if type(row[5]) == type('string'):
        item_list = divide(row[5])
        for i in range(len(item_list)):
            if len(item_list[i][0]) > 13:
                front = item_list[i][0][:13]
                back = item_list[i][0][13:]
                draw.text((col_item[0], row_item[i]-20), front, fill="black", font=smallFont, align='center') 
                draw.text((col_item[0], row_item[i]+20), back, fill="black", font=smallFont, align='center') 
            else:
                draw.text((col_item[0], row_item[i]), item_list[i][0], fill="black", font=smallFont, align='center') 
            for n in range(1,3):
                draw.text((col_item[n], row_item[i]), item_list[i][n], fill="black", font=smallFont, align='center') 

    for i in range(len(row[6])):
        check += 1
        file_id = row[6][i]
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = googleapiclient.http.MediaIoBaseDownload(fh, request)

        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("[{}] index:{}의 {}번째 사진! ".format(check, index, i)+"Download %d%%." % int(status.progress() * 100))

        add_image = Image.open(fh)
    
        if int(add_image.size[0]*(1300/add_image.size[1])) > 1300:
            target_image.paste(im = add_image.resize((1300, int(add_image.size[1]*(1300/add_image.size[0])))), box=location_receipt)              
        else:
            target_image.paste(im = add_image.resize((int(add_image.size[0]*(1300/add_image.size[1])), 1300)), box=location_receipt)
        target_image.save(path+"/result/use/{}월 {}반/영수증_{}_{}.jpg".format(month, when, index, i))
        
print(f'전체 영수증 개수: {whole_num}\n영수증 양식 제작 개수: {check}')

[1] index:0의 0번째 사진! Download 100%.
[2] index:1의 0번째 사진! Download 100%.
전체 영수증 개수: 234
영수증 양식 제작 개수: 2
